# Finding hyperparameters for each method for each dataset using gridsearch

## Compile all methods on toy example

In [5]:
using Revise
using ConvexHullPricing
using DataFrames
using Plots
using JLD2
using ProgressBars

N_ITER = 1
const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer

ConvexHullPricing.Optimizer

In [116]:
ThermalGen = ConvexHullPricing.Utilitaries.ThermalGen(
    MinRunCapacity = [6],
    MaxRunCapacity = [16],
    RampUp = [5],
    RampDown = [5],
    StartUp = [6],
    ShutDown = [6],
    UpTime = [1],
    DownTime = [1],
    NoLoadConsumption = [10],
    MarginalCost = [53],
    FixedCost = [30],
)
instance = ConvexHullPricing.Utilitaries.Instance(
    LostLoad = 3000,
    Load = [6 11 16 11 11 16 11 16 20],
    ThermalGen = ThermalGen
)
LP_Relax = UT.LP_Relaxation(instance)
ObjM = UT.Matching(instance).Obj
ValueAtInitialPoint = UT.fast_oracle(instance, LP_Relax)[1] .- ObjM

-925.0

In [9]:
#OPT.BundleLevelMethod(instance, LP_Relax, N_ITER, .9)
#OPT.BundleProximalMethod(instance, LP_Relax, N_ITER, .9, 1e-1)
#OPT.DAdaptation(instance, LP_Relax, N_ITER)
OPT.DowG(instance, LP_Relax, N_ITER)
OPT.OptimizedGradientMethod(instance, LP_Relax, N_ITER, 1e-5)
#OPT.EstimatedPolyak(instance, LP_Relax, N_ITER, 1.)
#OPT.SubgradientMethod(instance, LP_Relax, N_ITER, 1.)
#OPT.lastSubgradientMethod(instance, LP_Relax, N_ITER, 1.)
#OPT.StochasticSubgradientMethod(instance, LP_Relax, N_ITER, 1., 1)
#OPT.ColumnGeneration(instance, LP_Relax, N_ITER, 1e-5)
#OPT.ColumnAndRowGeneration(instance, N_ITER, 1e-5)

([79.56249444444444, 89.24999722222222, 88.0, 86.12499722222222, 84.24999722222222, 88.0, 84.24999722222222, 86.125, 2999.9999911111113], [[79.5625, 89.25, 88.0, 86.125, 84.25, 88.0, 84.25, 86.125, 3000.0], [79.56249444444444, 89.24999722222222, 88.0, 86.12499722222222, 84.24999722222222, 88.0, 84.24999722222222, 86.125, 2999.9999911111113]], Any[-332112.734275, -332112.9999244445])

In [102]:
OPT.BundleLevelMethod(instance, LP_Relax, 20, .98)[3][end] .- ObjM

-2.6193447411060333e-9

In [117]:
smooths = [1e-5]
outputs = Float64[]
for smooth in tqdm(smooths)
    iterates = OPT.FastGradientMethod(instance, LP_Relax, 2000, smooth)[2]
    push!(outputs, maximum([UT.fast_oracle(instance, x)[1] for x in iterates]) .- ObjM)
end

0.0%┣                                                ┫ 0/1 [00:00<00:00, -0s/it]


100.0%┣██████████████████████████████████████████┫ 1/1 [00:40<Inf:Inf, InfGs/it]
100.0%┣██████████████████████████████████████████┫ 1/1 [00:40<Inf:Inf, InfGs/it]


In [118]:
@info ValueAtInitialPoint
outputs

┌ Info: -925.0
└ @ Main c:\Users\Sofiane\Desktop\ConvexHullPricing\notebooks\hyperparameter_search.ipynb:1


1-element Vector{Float64}:
 -245.7965152527322

## Belgian dataset

In [119]:
BEinstances = []
for file in readdir("data\\belgian"; join=true)
    push!(BEinstances, UT.load_data(file))
end
instance = BEinstances[2]
LP_Relax = UT.LP_Relaxation(instance)
ObjM = UT.Matching(instance).Obj
ValueAtInitialPoint = UT.fast_oracle(instance, LP_Relax)[1] .- ObjM

-254389.44432210922

In [120]:
steps = [1e-5, 1e-3, 1e-1]
outputs = Float64[]
for step in tqdm(steps)
    iterates = OPT.FastGradientMethod(instance, LP_Relax, 50, step)[2]
    push!(outputs, maximum([UT.fast_oracle(instance, x)[1] for x in iterates]) .- ObjM)
end

0.0%┣                                                ┫ 0/3 [00:00<00:00, -0s/it]

In [ ]:
@info ValueAtInitialPoint
outputs

## Californian dataset

In [54]:
CAinstances = []
for file in readdir("data\\ca"; join=true)
    push!(CAinstances, UT.load_data(file))
end
instance = CAinstances[12]
LP_Relax = UT.LP_Relaxation(instance)
ObjM = UT.Matching(instance).Obj
ValueAtInitialPoint = UT.fast_oracle(instance, LP_Relax)[1] .- ObjM

-102527.21235322952

In [55]:
steps = [5e-4]
outputs = Float64[]
for step in tqdm(steps)
    iterates = OPT.StochasticSubgradientMethod(instance, LP_Relax, 50, step, 10)[2]
    push!(outputs, maximum([UT.fast_oracle(instance, x)[1] for x in iterates]) .- ObjM)
end

0.0%┣                                                ┫ 0/4 [00:00<00:00, -0s/it]


25.0%┣██████████▊                                ┫ 1/4 [04:01<Inf:Inf, InfGs/it]


50.0%┣███████████████████████                       ┫ 2/4 [08:19<16:38, 499s/it]


75.0%┣██████████████████████████████████▌           ┫ 3/4 [12:50<06:25, 385s/it]


100.0%┣█████████████████████████████████████████████┫ 4/4 [17:09<00:00, 343s/it]
100.0%┣█████████████████████████████████████████████┫ 4/4 [17:09<00:00, 343s/it]


In [57]:
@info ValueAtInitialPoint
outputs

┌ Info: -102527.21235322952
└ @ Main c:\Users\Sofiane\Desktop\ConvexHullPricing\notebooks\hyperparameter_search.ipynb:1


4-element Vector{Float64}:
 -102500.20404720306
 -102527.21235322952
 -102527.21235322952
 -102527.21235322952